In [3]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime, timedelta
#import matplotlib.pyplot as plt
#import matplotlib.image as mping
#from string import Template
#import math
#from matplotlib.figure import Figure
#from matplotlib.backends.backend_agg import FigureCanvas
#import plotly.graph_objects as go
import panel as pn
import hvplot.pandas
from bokeh.models import ColumnDataSource, Label, LabelSet, Range1d
from bokeh.plotting import figure, output_file, show
import panel.widgets as pnw
from panel.interact import interact, interactive, fixed, interact_manual
from panel import widgets
import param
#import holoviews as hv




# Panel Extension to prepare the web style
#pn.extension('plotly')
pn.extension(comms='ipywidgets')
#pn.extension('echarts')
pn.extension()
#pn.config.js_files  = {'deck': 'https://unpkg.com/deck.gl@~5.2.0/deckgl.min.js'}
#pn.config.css_files = ['https://api.tiles.mapbox.com/mapbox-gl-js/v0.44.1/mapbox-gl.css']


bootstrap = pn.template.BootstrapTemplate(title='Sharpe 3', header_color='white',header_background='blue' )

pn.config.sizing_mode = 'stretch_width'

disclaimer = (
    'DISCLAIMER: Sharpe3 provides data regarding public stock market. It does NOT recommend or advice for any investment decission.\nData showed are a mathematical model based in historical public information.'
)



#Risk Free at 0%
rf = 0


#Fortmat output
pct = lambda x: '{:.2%}'.format(x)
dig = lambda x: '{:.2f}'.format(x)
#exp = lambda x: math.exp(x)


# Timing
D=400
date_D_days_ago = datetime.now() - timedelta(days=D)
now = datetime.now()
start_date = date_D_days_ago.strftime('%F')
end_date = now.strftime('%F')

#Test porfolio
#portfolio =['PTON', 'TSLA', 'GM','SPY']
US_Index = ['^GSPC','^IXIC']

#Data from Yahoo Finance
def StockData(ticker, start_d, end_d):
    data = web.get_data_yahoo(ticker, start = start_d, end = end_d)
    price =  pd.DataFrame(data['Adj Close'])
    #volume = pd.DataFrame(data['Volume'])
    return price




# Pofolio Optimization and Efficient Frontier (TO REVIEW)
def P_Optimization(df):
    ind_er = df.pct_change().apply(lambda x: np.log(1+x)).mean().apply(lambda x: x*250)
    cov_matrix = df.pct_change().apply(lambda x: np.log(1+x)).cov()
    corr_matrix = df.pct_change().apply(lambda x: np.log(1+x)).corr()
    #ann_sd = df.pct_change().apply(lambda x: np.log(1+x)).std().apply(lambda x: x*np.sqrt(250))

    p_ret = [] # Define an empty array for portfolio returns
    p_vol = [] # Define an empty array for portfolio volatility
    p_weights = [] # Define an empty array for asset weights

    num_assets = len(df.columns)
    num_portfolios = 4000
    
    for portfolio in range(num_portfolios):
        weights = np.random.random(num_assets)
        weights = weights/np.sum(weights)
        weights= weights.round(3)
        p_weights.append(weights)
        returns = np.dot(weights, ind_er) # Returns are the product of individual expected returns of asset and its 
                                      # weights 
        p_ret.append(returns)
        var = cov_matrix.mul(weights, axis=0).mul(weights, axis=1).sum().sum()# Portfolio Variance
        sd = np.sqrt(var) # Daily standard deviation
        ann_sd = sd*np.sqrt(250) # Annual standard deviation = volatility
        p_vol.append(ann_sd)
    
    data = {'Returns':p_ret, 'Volatility':p_vol}

    for counter, symbol in enumerate(df.columns.tolist()):
        #print(counter, symbol)
        data[symbol+' weight'] = [w[counter] for w in p_weights]

    portfolios  = pd.DataFrame(data) #Dataframe of the 4000 portfolios created
    
    return portfolios

#Performace for each stock
def Performance(p):
    window = ['D','W','M','3M','6M','A']
    perform = pd.DataFrame(0, index = p.columns, columns=window)

    for i in window:    
        df = p.resample(i).last().pct_change().tail(2)
        perform[i] = df.iloc[0,:].apply(pct) 
    
    df = p.resample('Y').last().pct_change().tail(1)
    perform['YTD'] = df.iloc[0,:].apply(pct)
    print ('Performance calculated')
    return perform



def Core_Calculations(portfolio):

    price = StockData(portfolio,start_date,end_date )
    #%time
    #Cleaning data
    price = price.dropna(axis=1, how='all')
    #volume = volume.dropna(axis=1, how='all')
    
    #Performance calculaations
    perform = Performance(price)
    
    #Calculations for annual view
    ann_mean = price.pct_change().apply(lambda x: np.log(1+x)).mean().apply(lambda x: x*250)
    ann_std = price.pct_change().apply(lambda x: np.log(1+x)).std().apply(lambda x: x*np.sqrt(250))
    corr = price.pct_change().apply(lambda x: np.log(1+x)).corr()
    #cov  = price.pct_change().apply(lambda x: np.log(1+x)).cov()
    Sharpe = (ann_mean - rf)/ann_std

    #Optimization
    portfolios = P_Optimization(price)
    min_vol_port = portfolios.loc[portfolios['Volatility'].idxmin()]
    optimal_risky_port = portfolios.loc[((portfolios['Returns']- rf)/portfolios['Volatility']).idxmax()]

    #Aggregation of Returns, Volatility and Sharpes
    indice = portfolio+['Minimum Volatility','Maximum Sharpe ratio']
    df = pd.DataFrame(0, index = indice, columns = ['Return','Volatility','Sharpe'])
    for i in portfolio:
        df.loc[i,'Return'] = ann_mean[i]
        df.loc[i,'Volatility'] = ann_std[i]
        df.loc[i,'Sharpe'] = Sharpe[i]
    df.loc['Minimum Volatility','Return'] = min_vol_port['Returns']
    df.loc['Minimum Volatility','Volatility'] = min_vol_port['Volatility']
    df.loc['Maximum Sharpe ratio','Return'] = optimal_risky_port['Returns']
    df.loc['Maximum Sharpe ratio','Volatility'] = optimal_risky_port['Volatility']
    df.loc['Minimum Volatility','Sharpe'] = (min_vol_port['Returns'] - rf)/min_vol_port['Volatility']
    df.loc['Maximum Sharpe ratio','Sharpe'] = (optimal_risky_port['Returns'] - rf)/optimal_risky_port['Volatility']
    
    return price, df, corr, min_vol_port, optimal_risky_port,perform

def Stats(price):
    perf = price.calc_stats()
    st = perf.to_csv(sep=',',path=None)
    df = pd.read_csv(st, index_col='Stat',error_bad_lines=False, engine="python")
    return df

def Plot_Portfolio(df):
    p = figure(title='Risk / Reward view')
    df.index.name = 'stocks'
    source = ColumnDataSource(df)
    p.scatter(x='Volatility', y='Return', size=8, source=df) 
    p.xaxis[0].axis_label = 'Volatility (log)'
    p.yaxis[0].axis_label = 'Return (log)'
    labels = LabelSet(x='Volatility', y='Return', text='stocks', level='glyph',
              x_offset=5, y_offset=5, source=source, render_mode='canvas')
    p.add_layout(labels)
    return p






def perf_plot(price):
    
    p1 = figure(height=300,name='Daily')
    p1 = price.hvplot.line()

    p2 = figure(height=300,name='MACD')
    MA = price.rolling(50).mean()
    MA.columns = MA.columns+'_MA'
    final = pd.concat([price,MA],axis=1)
    #final = final / final.iloc[0]
    p2 = final.hvplot.line()
    
    p3 = figure(height=300,name='vs_index')
    US_df = StockData(US_Index,start_date,end_date )
    US_df = US_df.dropna(axis=1, how='all')
    US_df = US_df / US_df.iloc[0]
    US_df.columns = ['SP500','NASDAQ']
    vs_index = pd.concat([price,US_df],axis=1)
    p3 = vs_index.hvplot.line()
    
    tabs = pn.Tabs(('Daily',p1), ('MACD',p2),('vs_US_index',p3))
    return tabs


class ActionExample(param.Parameterized):
    """
    Demonstrates how to use param.Action to trigger an update.
    """

    new_stock = param.String(default='TSLA,PTON')
    
    action = param.Action(lambda x: x.param.trigger('action'), label='Run Analytics')
        
    @param.depends('action')
    def get_P(self):
        return self.new_stock
  
   # method is watching whether model_trained is updated
    @param.depends('new_stock')
    def update_graph(self):
        if self.new_stock:
            new_portfolio = self.new_stock
            new_portfolio = new_portfolio.split(',')
            price, df, corr,mv_p,or_p,perform = Core_Calculations(new_portfolio)
            perform_pane = pn.pane.DataFrame(perform)
            new_df = pd.concat([or_p,mv_p], axis=1)
            columns_name = ['Maximum Sharpe Ratio', 'Minimum Volatility']
            new_df.columns = columns_name
            new_df = new_df.drop('Returns',axis=0)
            new_df = new_df.drop('Volatility',axis=0)
            price = price / price.iloc[0]
            #price_plot = price.hvplot.line()
            tab_plot = perf_plot(price)
            new_plot = Plot_Portfolio(df)
            df['Return'] = df['Return'].apply(pct)
            df['Volatility'] = df['Volatility'].apply(pct)
            df['Sharpe'] = df['Sharpe'].apply(dig)
            df = df.sort_values(by='Sharpe', ascending = False)
            df_pane = pn.pane.DataFrame(df)
            corr_pane = pn.pane.DataFrame(corr)
            div_index = dig(abs(corr.iloc[:,0].sum()-1)/(len(corr.index)))
            opt_pane = pn.pane.DataFrame(new_df)
            return pn.Column('<br><h2>Performance</h2>',
                             tab_plot,'<br>',
                             perform_pane,'<br>',
                             pn.layout.Divider(),
                             '<br><h2>Correlation Matrix</h2>',
                             corr_pane,
                             '<br><p>Diversification Index = '+div_index+'</p> The lower the number, the more diversed is the portfolio',
                             pn.layout.Divider(),
                             '<br><h2>Portfolio Optimization</h2>', 
                             df_pane,'<br>',
                             new_plot,
                             '<br><p>Weights Portfolio</p>',opt_pane,'<br>',
                             pn.layout.Divider(),
                             pn.layout.Divider(),
                            pn.pane.Alert(disclaimer, alert_type="primary"))
        else:
            return "Model not trained yet"

        
        
action_example = ActionExample()

message = 'eg. CSCO,PTON,TSLA'
bootstrap.header.append('        Data-Driven Investing')
bootstrap.sidebar.append(message)
bootstrap.sidebar.append(pn.panel(action_example.param, show_labels=False, show_name=False, margin=0))
#bootstrap.sidebar.append(action_example.get_P)

bootstrap.main.append(pn.Column(action_example.update_graph))
#%time
bootstrap.show()
#bootstrap.servable()

Performance calculated
Launching server at http://localhost:55445


In [41]:
df1 = pd.DataFrame({'key' : ['a','b','c'],'value': range(3)}, index = ['a1','a2','a3'])
df2 = pd.DataFrame({'key' : ['x','y','x','v'],'value': range(4)}, index = ['a1','a2','a3','a4'])

In [42]:
df1

,key,value
a1,a,0
a2,b,1
a3,c,2


In [44]:
df2

,key,value
a1,x,0
a2,y,1
a3,x,2
a4,v,3


In [59]:

#df1 in from Tupper. df2 is from input
def Join_Df(df1, df2):
    for i in df2.index:
        if df1.index.str.match(i).any()== True:
            df1.loc[i,'key'] = df2.loc[i,'key']
            print (i)
        else:
            df1 = df1.append(df2.loc[i,:])
            print(i)
    return df1
            

a1
a2
a3
a4


In [60]:
df1

,key,value
a1,x,0
a2,y,1
a3,x,2
a4,v,3


In [58]:
pd.merge(df,d2,)

,key,value
a1,x,0
a2,y,1
a3,x,2
